In [20]:
global max_bit 
max_bit = 10
max_node = 2**max_bit

In [2]:
import hashlib

In [3]:
def gethash(key):
    key = key.encode()
    h = hashlib.sha1(key)
    return int(h.hexdigest(),16)%max_node

In [4]:
key_list=[]
for i in range(max_node):
    h = gethash("%s"%i)
    key_list.append(h)

In [40]:
class ChordNode:
    def __init__(self,id):
        self.id = gethash(id)
        self.successor = None
        self.predecessor = None
        self.finger_table = {}
        self.key = []
    
    def find_successor(self,keyid):#find keyid successor
        keyid = gethash(keyid)
        if keyid == self.id: 
            #case 1:key_id = node_id
            return self
        if self == self.successor: 
            #case 2:only one node
            return self
        if self.successor.id >= keyid and self.id < keyid:
            #case 3:node_id<key_id<successor_id
            return self.successor
        if self.id > self.successor.id and keyid > self.id:
            #case 4:successor_id<node_id<key_id
            return self.successor
        if self.id > self.successor.id and keyid <= self.successor.id:
            #case 5:key_id<successor_id<self.id
            return self.successor
        else:
            predecessor = self.predecessor(keyid)
            if predecessor == self:
                return predecessor.successor.find_successor(keyid)
            else:
                return predecessor.find_successor(keyid)
    
    def predecessor(self,keyid):
        for i in range(max_bit,-1,-1):
            if self.finger_table[i].id >= self.id and self.finger_table[i].id < keyid:
                return self.finger_table[i]      
        return self
    
#     def reset_finger_table(self):
#         for i in range(max_bit):
#             self.finger_table[i] = self.successor

    def join(self,newNode):
        if newNode:
            self.init_finger_table(newNode)
            self.update_others
        else:
            #only one node in the network
            for i in range(max_bit):
                self.finger_table[i] = self
            newNode.predecessor = self
    
    def init_finger_table(self):
        return None
    
    def update_finger_table(self):
        return None
    
    def update_others(self):
        return None